In [1]:
import dataextraction as de
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re,string,unicodedata
from string import punctuation
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rohang62/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = de.extract_data('data/train.jsonl')
train = de.parse_json(data, True)

data = de.extract_data('data/test.jsonl')
test = de.parse_json(data, False)

In [3]:
print(len(train.index))
print(len(test.index))

5000
1800


In [4]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [5]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i != "@USER":
            final_text.append(i.strip().lower())
    return " ".join(final_text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

train['response']=train['response'].apply(denoise_text)
test['response']=test['response'].apply(denoise_text)

In [6]:
train.head()

,label,response
0,SARCASM,get .. obviously care would've moved right alo...
0,SARCASM,trying protest talking labels label wtf make em
0,SARCASM,makes insane money movies einstein #learnhowth...
0,SARCASM,meanwhile trump even release sat scores wharto...
0,SARCASM,pretty sure anti-lincoln crowd claimed democra...


In [108]:
with open('train_ft.txt', 'w') as f:
    for each_text, each_label in zip(train['response'], train['label']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [153]:
import fasttext
model = fasttext.train_supervised('train_ft.txt', epoch=300, lr=1, wordNgrams=5, verbose=2, minCount=1)

In [154]:
def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{precision}')
    print(f'{recall}')

print_results(*model.test('train_ft.txt'))

1.0
1.0


In [155]:
def predict_is_sarcastic(text):
    return model.predict(text, k=2)

In [156]:
!rm answer.txt

In [157]:
with open('answer.txt', 'w') as f:
    for each_id, each_text in zip(test['id'], test['response']):
        labels, probs = predict_is_sarcastic(f'{each_text}')
        l = None
        if (probs[0] > probs[1] and labels[0] == "__label__SARCASM"):
            l = "SARCASM"
        elif (probs[0] > probs[1] and labels[0] != "__label__SARCASM"): 
            l = "NOT_SARCASM"
        elif (probs[0] < probs[1] and labels[0] == "__label__SARCASM"): 
            l = "NOT_SARCASM"
        else:
            l = "SARCASM"
        f.writelines(f'{each_id},{l}\n')

In [98]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [99]:
train = train.replace("SARCASM", 1)
train = train.replace("NOT_SARCASM", 0)

In [100]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [101]:
tokenized = train['response'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [102]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [103]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5000, 73)

In [104]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [105]:
train_features = last_hidden_states[0][:,0,:].numpy()
train_labels = train['label']

In [106]:
tokenized = test['response'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [107]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [108]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1800, 49)

In [109]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [110]:
lr_clf = LogisticRegression(max_iter = 10000)
lr_clf.fit(train_features, train_labels)

LogisticRegression(max_iter=10000)

In [111]:
lr_clf.score(train_features, train_labels)

0.8128

In [112]:
test_features = last_hidden_states[0][:,0,:].numpy()
test_pred = lr_clf.predict(test_features)

In [324]:
from numpy import savetxt
savetxt('train_features.csv', train_features, delimiter=',')
savetxt('train_labels.csv', train_labels.to_numpy(), delimiter=',')
savetxt('test_features.csv', train_labels.to_numpy(), delimiter=',')

In [375]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 30)
        self.fc2 = nn.Linear(30, n_features)
        self.fc3 = nn.Linear(n_features, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))


net = Net(train_features.shape[1])

In [376]:
params = list(net.parameters())

In [377]:
X_train = torch.from_numpy(train_features).float()
y_train = torch.squeeze(torch.from_numpy(train_labels.to_numpy()).float())
X_test = torch.from_numpy(test_features).float()

In [378]:
criterion = nn.BCELoss()

In [379]:
optimizer = optim.Adam(net.parameters(), lr=1e-4, weight_decay = 0.001)

In [380]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
net = net.to(device)
criterion = criterion.to(device)

In [381]:
def calculate_accuracy(y_true, y_pred):
    predicted = y_pred.ge(.5).view(-1)
    return (y_true == predicted).sum().float() / len(y_true)

In [382]:
epochs = 1000
def round_tensor(t, decimal_places=3):
    return round(t.item(), decimal_places)
for epoch in range(epochs):
    y_pred = net(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    if epoch % (epochs / 10) == 0:
        train_acc = calculate_accuracy(y_train, y_pred)
        print(f'''epoch {epoch} Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}''')
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

epoch 0 Train set - loss: 0.693, accuracy: 0.486
epoch 20 Train set - loss: 0.681, accuracy: 0.709
epoch 40 Train set - loss: 0.653, accuracy: 0.721
epoch 60 Train set - loss: 0.612, accuracy: 0.727
epoch 80 Train set - loss: 0.569, accuracy: 0.73
epoch 100 Train set - loss: 0.54, accuracy: 0.733


KeyboardInterrupt: 

In [373]:
y_test = net(X_test)
print(y_test)
y_labels = []
for i in range(y_test.shape[0]):
    if y_test[i] < 0.5:
        y_labels.append("NOT_SARCASM")
    else:
        y_labels.append("SARCASM")

tensor([[0.2901],
        [0.7361],
        [0.8920],
        ...,
        [0.4301],
        [0.1187],
        [0.1857]], grad_fn=<SigmoidBackward>)


In [374]:
with open('answer.txt', 'w') as f:
    for tid, pred in zip(test['id'], y_labels):
        f.writelines(f'{tid},{pred}\n')